In [37]:
import requests
import csv
import time

# ═══════════════════════════════════════════════════════════════════════════════
# SCRIPT DE EXTRACCIÓN DE PRODUCTOS DE MERCADONA
# ═══════════════════════════════════════════════════════════════════════════════

# --- CONFIGURACIÓN ---
BASE_URL = "https://tienda.mercadona.es/api/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept": "application/json"
}
CSV_FILE = "productos_mercadona.csv"
DELAY_BETWEEN_REQUESTS = 0.3  # segundos

# --- VARIABLES GLOBALES ---
productos_guardados = set()  # IDs de productos únicos
total_productos = 0

# ═══════════════════════════════════════════════════════════════════════════════
# FUNCIONES DE API
# ═══════════════════════════════════════════════════════════════════════════════

def obtener_datos(url):
    """
    Hace una petición GET a la URL especificada.
    Retorna el JSON o None si hay error.
    """
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        respuesta = requests.get(url, headers=HEADERS, timeout=10)
        respuesta.raise_for_status()
        return respuesta.json()
    except requests.exceptions.RequestException as error:
        print(f"⚠️  Error al obtener {url}: {error}")
        return None


def obtener_categorias_principales():
    """
    Obtiene todas las categorías principales de Mercadona.
    Retorna una lista de categorías.
    """
    url = f"{BASE_URL}categories/"
    datos = obtener_datos(url)
    if datos and "results" in datos:
        return datos["results"]
    return []


def procesar_categoria(categoria_id, ruta_categorias, csv_writer):
    """
    Procesa una categoría de forma recursiva.
    
    Estructura de la API:
    - GET /categories/{id} retorna un objeto con un array "categories"
    - Cada elemento de "categories" es una subcategoría que puede tener "products"
    - Los productos están en: subcategoria["products"]
    
    Args:
        categoria_id: ID de la categoría a procesar
        ruta_categorias: Lista con la ruta de categorías padre
        csv_writer: Writer del CSV para guardar productos
    """
    global total_productos
    
    url = f"{BASE_URL}categories/{categoria_id}"
    datos = obtener_datos(url)
    
    if not datos:
        return
    
    # Verificar si tiene subcategorías
    if "categories" not in datos:
        return
    
    subcategorias = datos["categories"]
    if not subcategorias:
        return
    
    # Procesar cada subcategoría
    for subcategoria in subcategorias:
        subcat_id = subcategoria.get("id")
        subcat_nombre = subcategoria.get("name")
        
        if not subcat_id or not subcat_nombre:
            continue
        
        # Construir ruta completa de la categoría
        ruta_completa = ruta_categorias + [subcat_nombre]
        ruta_str = " > ".join(ruta_completa)
        
        print(f"\n📁 {ruta_str} (ID: {subcat_id})")
        
        # IMPORTANTE: Extraer productos DIRECTAMENTE de esta subcategoría
        if "products" in subcategoria:
            productos = subcategoria["products"]
            
            if productos:  # Solo si hay productos
                print(f"   ✅ {len(productos)} productos encontrados")
                
                # Procesar cada producto
                for producto in productos:
                    guardar_producto(producto, ruta_str, csv_writer)
                    total_productos += 1
            else:
                print(f"   ℹ️  Sin productos")
        else:
            print(f"   ℹ️  Sin productos")
        
        # Llamada recursiva para procesar subcategorías anidadas dentro de esta
        procesar_categoria(subcat_id, ruta_completa, csv_writer)


def guardar_producto(producto, ruta_categoria, csv_writer):
    """
    Guarda un producto en el CSV si no ha sido guardado antes.
    También lo imprime en consola.
    
    Args:
        producto: Diccionario con los datos del producto
        ruta_categoria: String con la ruta completa de categorías
        csv_writer: Writer del CSV
    """
    producto_id = producto.get("id")
    
    # Evitar duplicados
    if not producto_id or producto_id in productos_guardados:
        return
    
    # Extraer datos del producto
    nombre = producto.get("display_name", "")
    packaging = producto.get("packaging", "")
    
    # Extraer información de precios
    price_info = producto.get("price_instructions", {})
    precio_unidad = price_info.get("unit_price", "")
    precio_bulk = price_info.get("bulk_price", "")
    precio_referencia = price_info.get("reference_price", "")
    formato_referencia = price_info.get("reference_format", "")
    
    # Construir string de precio de referencia
    precio_ref_completo = f"{precio_referencia} {formato_referencia}".strip()
    
    # Imprimir en consola (como en el PHP)
    print(f"      • ID {producto_id}: {nombre} ({precio_unidad}€)")
    
    # Escribir en CSV
    csv_writer.writerow({
        "id": producto_id,
        "nombre": nombre,
        "subtitulo": packaging,
        "precio_unidad": precio_unidad,
        "precio_unidad_por": precio_bulk,
        "precio_referencia": precio_ref_completo,
        "ruta_categoria": ruta_categoria
    })
    
    # Marcar como guardado
    productos_guardados.add(producto_id)


# ═══════════════════════════════════════════════════════════════════════════════
# EJECUCIÓN PRINCIPAL
# ═══════════════════════════════════════════════════════════════════════════════

def main():
    """Función principal que ejecuta el proceso completo de extracción."""
    
    print("╔" + "═" * 78 + "╗")
    print("║" + " " * 20 + "EXTRACTOR DE PRODUCTOS MERCADONA" + " " * 26 + "║")
    print("╚" + "═" * 78 + "╝")
    print()
    
    # Definir columnas del CSV
    columnas = [
        "id",
        "nombre",
        "subtitulo",
        "precio_unidad",
        "precio_unidad_por",
        "precio_referencia",
        "ruta_categoria"
    ]
    
    try:
        # Abrir archivo CSV
        with open(CSV_FILE, 'w', newline='', encoding='utf-8') as archivo_csv:
            writer = csv.DictWriter(archivo_csv, fieldnames=columnas, delimiter=';')
            writer.writeheader()
            
            # Obtener categorías principales
            print("🔍 Obteniendo categorías principales...\n")
            categorias = obtener_categorias_principales()
            
            if not categorias:
                print("❌ No se pudieron obtener las categorías")
                return
            
            print(f"✅ {len(categorias)} categorías principales encontradas\n")
            
            # Procesar cada categoría principal
            for categoria in categorias:
                cat_id = categoria.get("id")
                cat_nombre = categoria.get("name")
                
                if not cat_id or not cat_nombre:
                    continue
                
                print("\n" + "═" * 80)
                print(f"🏷️  {cat_nombre.upper()} (ID: {cat_id})")
                print("═" * 80)
                
                # Procesar esta categoría y todas sus subcategorías
                procesar_categoria(cat_id, [cat_nombre], writer)
            
            # Resumen final
            print("\n" + "╔" + "═" * 78 + "╗")
            print("║" + " " * 30 + "PROCESO COMPLETADO" + " " * 30 + "║")
            print("╠" + "═" * 78 + "╣")
            print(f"║  📊 Total de productos encontrados: {total_productos:<44} ║")
            print(f"║  💾 Productos únicos guardados:     {len(productos_guardados):<44} ║")
            print(f"║  📄 Archivo generado:                {CSV_FILE:<44} ║")
            print("╚" + "═" * 78 + "╝")
            
    except Exception as error:
        print(f"\n❌ Error inesperado: {error}")
        import traceback
        traceback.print_exc()


# Ejecutar el script
if __name__ == "__main__":
    main()


╔══════════════════════════════════════════════════════════════════════════════╗
║                    EXTRACTOR DE PRODUCTOS MERCADONA                          ║
╚══════════════════════════════════════════════════════════════════════════════╝

🔍 Obteniendo categorías principales...

✅ 26 categorías principales encontradas


════════════════════════════════════════════════════════════════════════════════
🏷️  ACEITE, ESPECIAS Y SALSAS (ID: 12)
════════════════════════════════════════════════════════════════════════════════
⚠️  Error al obtener https://tienda.mercadona.es/api/categories/12: 404 Client Error: Not Found for url: https://tienda.mercadona.es/api/categories/12

════════════════════════════════════════════════════════════════════════════════
🏷️  AGUA Y REFRESCOS (ID: 18)
════════════════════════════════════════════════════════════════════════════════
⚠️  Error al obtener https://tienda.mercadona.es/api/categories/18: 404 Client Error: Not Found for url: https://tienda.mercadona

In [ ]:
import requests
import json

# PRUEBA CON CATEGORÍA 45 - DEBUG DETALLADO

url = "https://tienda.mercadona.es/api/categories/49"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept": "application/json"
}

print("🔍 Haciendo petición a:", url)
print("=" * 80)

response = requests.get(url, headers=headers, timeout=10)
print(f"✅ Status Code: {response.status_code}")

data = response.json()

print(f"\n📦 Claves principales en la respuesta:")
print(f"   {list(data.keys())}")

print(f"\n📋 Información de la categoría:")
print(f"   ID: {data.get('id')}")
print(f"   Nombre: {data.get('name')}")

if "categories" in data:
    subcategorias = data["categories"]
    print(f"\n✅ Tiene {len(subcategorias)} subcategoría(s)")
    
    for i, subcat in enumerate(subcategorias):
        print(f"\n   {'─' * 70}")
        print(f"   📁 Subcategoría {i+1}:")
        print(f"      ID: {subcat.get('id')}")
        print(f"      Nombre: {subcat.get('name')}")
        print(f"      Claves: {list(subcat.keys())}")
        
        if "products" in subcat:
            products = subcat["products"]
            print(f"      ✅ PRODUCTOS ENCONTRADOS: {len(products)}")
            
            # Mostrar los primeros 3 productos
            for j, product in enumerate(products[:3]):
                prod_id = product.get("id")
                prod_name = product.get("display_name")
                price_info = product.get("price_instructions", {})
                price = price_info.get("unit_price", "N/A")
                
                print(f"         {j+1}. ID {prod_id}: {prod_name} ({price}€)")
            
            if len(products) > 3:
                print(f"         ... y {len(products) - 3} productos más")
        else:
            print(f"      ⚠️ NO tiene clave 'products'")
            
        # Verificar si tiene subcategorías anidadas
        if "categories" in subcat:
            print(f"      🔄 Tiene subcategorías anidadas: {len(subcat['categories'])}")
else:
    print("\n❌ NO tiene clave 'categories' en la respuesta principal")

print("\n" + "=" * 80)
print("✅ DEBUG COMPLETADO")


╔══════════════════════════════════════════════════════════════════════════════╗
║                         DEBUG TODAS LAS CATEGORÍAS                           ║
╚══════════════════════════════════════════════════════════════════════════════╝

🔍 Obteniendo categorías principales...

✅ 26 categorías principales encontradas


🏷️  [1/26] ACEITE, ESPECIAS Y SALSAS (ID: 12)
🔍 Haciendo petición a: https://tienda.mercadona.es/api/categories/12
✅ Status Code: 404
⚠️  Error: Status Code 404

🏷️  [2/26] AGUA Y REFRESCOS (ID: 18)
🔍 Haciendo petición a: https://tienda.mercadona.es/api/categories/18
✅ Status Code: 404
⚠️  Error: Status Code 404

🏷️  [3/26] APERITIVOS (ID: 15)
🔍 Haciendo petición a: https://tienda.mercadona.es/api/categories/15
✅ Status Code: 404
⚠️  Error: Status Code 404

🏷️  [4/26] ARROZ, LEGUMBRES Y PASTA (ID: 13)
🔍 Haciendo petición a: https://tienda.mercadona.es/api/categories/13
✅ Status Code: 404
⚠️  Error: Status Code 404

🏷️  [5/26] AZÚCAR, CARAMELOS Y CHOCOLATE (ID: 9)
🔍 

╔══════════════════════════════════════════════════════════════════════════════╗
║                    ANÁLISIS COMPLETO DE CATEGORÍAS MERCADONA                 ║
╚══════════════════════════════════════════════════════════════════════════════╝

🔍 Obteniendo categorías principales...
✅ 26 categorías principales encontradas


🏷️  [1/26] ACEITE, ESPECIAS Y SALSAS (ID: 12)
⚠️  Error: Status Code 404

🏷️  [2/26] AGUA Y REFRESCOS (ID: 18)
⚠️  Error: Status Code 404

🏷️  [3/26] APERITIVOS (ID: 15)
⚠️  Error: Status Code 404

🏷️  [4/26] ARROZ, LEGUMBRES Y PASTA (ID: 13)
⚠️  Error: Status Code 404

🏷️  [5/26] AZÚCAR, CARAMELOS Y CHOCOLATE (ID: 9)
⚠️  Error: Status Code 404

🏷️  [6/26] BEBÉ (ID: 24)
⚠️  Error: Status Code 404

🏷️  [7/26] BODEGA (ID: 19)
⚠️  Error: Status Code 404

🏷️  [8/26] CACAO, CAFÉ E INFUSIONES (ID: 8)
⚠️  Error: Status Code 404

🏷️  [9/26] CARNE (ID: 3)
⚠️  Error: Status Code 404

🏷️  [10/26] CEREALES Y GALLETAS (ID: 7)
⚠️  Error: Status Code 404

🏷️  [11/26] CHARCUTERÍA Y 